In [2]:
import pandas as pd
df = pd.read_csv("parlaspeech.csv")
df.columns

Index(['file', 'start', 'end', 'words', 'word_start_times', 'word_end_times',
       'asr', 'true_file', 'reconame', 'longname', 'yt_hash', 'hashname',
       'key', 'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',
       'split'],
      dtype='object')

In [3]:
df_pruned = df.loc[:, ['file', 'start', 'end',
     'true_file', 'reconame', 'yt_hash', 'hashname',
     'was_in_samples', 'human_transcript', 'asr_transcript', 'sim',
       'split']]

In [4]:
df_pruned.to_csv("parlaspeech_pruned.csv", index=False)

In [5]:
df_pruned.to_json("parlaspeech_pruned.json", orient="records", lines=True)

In [6]:
c1 = df_pruned.split == "test"
c2 = df_pruned.split == "dev"
available = 224031104 
corpus_size = 1316725396

train_fraction = available/corpus_size - (sum(c1) + sum(c2))/df_pruned.shape[0]
train_fraction

0.10467166622109986

Zaokrožim na 0.1 

In [7]:
keep_df = pd.concat([
    df_pruned.loc[c1, :],
    df_pruned.loc[c2, :],
    df_pruned.loc[(~c1)&(~c2), :].sample(frac=0.1, random_state=42)
])
keep_df.to_csv("keep_pruned.csv", index=False)

In [15]:
from shutil import copyfile
import os
for file in keep_df.hashname:
    copyfile(os.path.join("/home/rupnik/macocu/task8/data", file),
            os.path.join("/home/rupnik/macocu/task8/transfer", file)
            )